In [58]:
import pandas as pd
from sklearn.base import TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, RobustScaler, FunctionTransformer


SyntaxError: expected ':' (558899375.py, line 8)

Your Client FinMan is a financial services company that provides various financial services like loan, investment funds, insurance etc. to its customers. FinMan wishes to cross-sell health insurance to the existing customers who may or may not hold insurance policies with the company. The company recommend health insurance to it's customers based on their profile once these customers land on the website. Customers might browse the recommended health insurance policy and consequently fill up a form to apply. When these customers fill-up the form, their Response towards the policy is considered positive and they are classified as a lead.

Once these leads are acquired, the sales advisors approach them to convert and thus the company can sell proposed health insurance to these leads in a more efficient manner.

## Erstelle aus `health-insurance.csv` einen DataFrame

In [45]:
insurance = pd.read_csv('./data/health-insurance.csv')
insurance.head()

,ID,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium,Response
0,1,C3,3213,Rented,Individual,36,36,No,X1,14+,3.0,22,11628.0,0
1,2,C5,1117,Owned,Joint,75,22,No,X2,NaN,NaN,22,30510.0,0
2,3,C5,3732,Owned,Individual,32,32,No,NaN,1.0,1.0,19,7450.0,1
3,4,C24,4378,Owned,Joint,52,48,No,X1,14+,3.0,19,17780.0,0
4,5,C8,2190,Rented,Individual,44,44,No,X2,3.0,1.0,16,10404.0,0


In [43]:
insurance["Health Indicator"].unique()

array(['X1', 'X2', nan, 'X4', 'X3', 'X6', 'X5', 'X8', 'X7', 'X9'],
      dtype=object)

[1.0,
 2.0,
 3.0,
 4.0,
 5.0,
 6.0,
 7.0,
 8.0,
 9.0,
 10.0,
 11.0,
 12.0,
 13.0,
 14.0,
 15.0,
 16.0,
 17.0,
 18.0,
 19.0,
 20.0,
 21.0,
 22.0,
 23.0,
 24.0,
 25.0,
 26.0,
 27.0,
 28.0,
 29.0,
 30.0,
 31.0,
 32.0,
 33.0,
 34.0,
 35.0,
 36.0]

In [57]:
columnTransformer = ColumnTransformer(
    transformers=[("oneHotEncoder", OneHotEncoder(), ["Is_Spouse", "Reco_Insurance_Type", "Accomodation_Type"]),
                  ("extractNumbers", FunctionTransformer(func=lambda x: x.extract("\d+")), ["Health Indicator", "City_Code"])],
    remainder="passthrough")



pipe = Pipeline([("transformers", columnTransformer),
                 #("scalar", RobustScaler())
                 ])


insurance = pipe.fit_transform(insurance)
insurance

AttributeError: 'DataFrame' object has no attribute 'extract'

## Data Cleaning

* fehlende `Holding_Policy_Duration`/`Holding_Policy_Duration`-Werte sind gleich viele, sollten also gleich behandelt werden
* betrachte die Ausprägungen von `Health Indicator`. Auch hier gibt es einige `NaN`s

## Training
* 20% Testdaten
* `random_state=42` sofern unterstützt
* target feature: `Response`
* Überlege, welche Metrik maximiert werden soll (*accuracy*/*recall*/*f1*) und was die Business Implikation des erhaltenen Modells ist, diese Metrik geht in die `GridSearchCV` als `scoring` ein
* Trainiere mindestens
    * eine optimierte `LogisticRegression` mit `GridSearchCV` (optimiere `solver`, `C` und `class_weight`)
    * einen optimierten `KNeighborsClassifier` mit `GridSearchCV` (optimiere `n_neighbors` und `weights`)
    * zwei ANNs (eins mit `class_weights` und eins ohne).
        `compile(metrics=[<metrik>]` kann entweder ein String übergeben werden oder eine Instanz einer Klasse aus `keras.metrics`

## Evaluiere die Modelle und Interpretiere deine Lösung bezüglich *precision*/*recall*/*f1*